In [1]:
# Import the Pandas library

import openpyxl
import io
import tempfile
import pandas as pd
import numpy as np
import sys

sys.path.append('../')

from models.linear_regression import LinReg

lin_reg = LinReg()
path = "../.test_data/dataprocess.xlsx"

coded_df = lin_reg.process_excel(path)


print(coded_df.info())
coded_df

<class 'pandas.core.frame.DataFrame'>
Index: 3023 entries, 0 to 3022
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   NAME             3023 non-null   object        
 1   date             3023 non-null   datetime64[ns]
 2   color_and_value  3023 non-null   object        
 3   day_of_year      3023 non-null   int32         
 4   name_as_number   3023 non-null   int64         
 5   value            3023 non-null   float64       
 6   color            3023 non-null   object        
 7   color_code       3023 non-null   int64         
 8   next_color_code  3023 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(3)
memory usage: 224.4+ KB
None


,NAME,date,color_and_value,day_of_year,name_as_number,value,color,color_code,next_color_code
0,GJ1,2024-12-07,-16.155 | 00FFFF,342,1,-16.15500,00FFFF,0,0.0
1,GJ2,2024-12-07,-10.131 | 00FFFF,342,2,-10.13100,00FFFF,0,1.0
2,GJ3,2024-12-07,6.719 | 00FF00,342,3,6.71900,00FF00,1,0.0
3,GJ4,2024-12-07,8.2 | 00FFFF,342,4,8.20000,00FFFF,0,1.0
4,GJ5,2024-12-07,-7.444 | 00FF00,342,5,-7.44400,00FF00,1,0.0
...,...,...,...,...,...,...,...,...,...
3018,GJ4,2024-04-16,-12.2896 | 8EA9DB,107,4,-12.28960,8EA9DB,8,8.0
3019,GJ5,2024-04-16,-3.38613 | 8EA9DB,107,5,-3.38613,8EA9DB,8,3.0
3020,GJ6,2024-04-16,-9.74303 | FFFF00,107,6,-9.74303,FFFF00,3,5.0
3021,GJ7,2024-04-16,-10.9962 | A9D08E,107,7,-10.99620,A9D08E,5,7.0


In [2]:
lin_reg.color_mapping



{0: '00FFFF',
 1: '00FF00',
 2: 'FF9900',
 3: 'FFFF00',
 4: 'D5A6BD',
 5: 'A9D08E',
 6: '9BC2E6',
 7: 'FFC000',
 8: '8EA9DB',
 9: 'FF0000'}

In [3]:
lin_reg.last_df

,NAME,2024-04-16 00:00:00
0,GJ1,-19.8579 | FFFF00
1,GJ2,-10.1783 | FFFF00
2,GJ3,-7.68448 | FFFF00
3,GJ4,-12.2896 | 8EA9DB
4,GJ5,-3.38613 | 8EA9DB
5,GJ6,-9.74303 | FFFF00
6,GJ7,-10.9962 | A9D08E
7,GJ8,16.04017 | FFC000
8,GJ9,-5.71004 | 8EA9DB


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = lin_reg.train_test_split(coded_df)


print(X_train)
print(y_train)

[[ 8.200000e+01  5.000000e+00  4.109346e+00  7.000000e+00]
 [ 1.380000e+02  7.000000e+00 -1.890800e+01  3.000000e+00]
 [ 1.660000e+02  6.000000e+00  9.540950e+00  3.000000e+00]
 ...
 [ 1.610000e+02  6.000000e+00  1.164450e-01  7.000000e+00]
 [ 1.880000e+02  8.000000e+00  6.156937e+01  7.000000e+00]
 [ 1.180000e+02  6.000000e+00 -6.862510e+00  5.000000e+00]]
[8. 6. 5. ... 6. 3. 5.]


In [5]:
# Create and train the model

lin_reg.fit(path)

print(f'mse : {lin_reg.mse}' )

print(f'accuracy_percentage : {lin_reg.accuracy_percentage}' )

mse : 3.0858085808580857
accuracy_percentage : 0.19801980198019803


In [6]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


# Predicting new values (You would add new 'day_of_year' values here to make predictions)
predicted_colors = lin_reg.regressor.predict(X_test)

# Actual vs Predicted colors
mse = mean_squared_error(y_test, predicted_colors)


print(f'Mean Squared Error (MSE): {mse}')


Mean Squared Error (MSE): 3.0939380686327937


In [7]:

predicted_colors =  [round(x) for x in predicted_colors]

# Calculate the number of correct predictions
correct_predictions = sum(y_test == predicted_colors)
# Calculate the total number of predictions
total_predictions = len(predicted_colors)
# Calculate the percentage of correct predictions
accuracy_percentage = (correct_predictions / total_predictions) * 100
# Calculate the percentage of incorrect predictions
incorrect_percentage = 100 - accuracy_percentage

# Print the values
print(f'Percentage of correct predictions: {accuracy_percentage}%')
print(f'Percentage of incorrect predictions: {incorrect_percentage}%')


Percentage of correct predictions: 19.801980198019802%
Percentage of incorrect predictions: 80.1980198019802%


In [8]:
#make predictions

